In [2]:
import alpaca_backtrader_api
import backtrader as bt
from datetime import datetime

In [3]:
ALPACA_API_KEY ="PK8SSSXD3FX164IYREJ5"
ALPACA_SECRET_KEY ="NPKEgtQyt3fsRodbOqn4d46V19XI5W9S8Cv64dDT"
ALPACA_PAPER = True

In [ ]:
class SmaCross(bt.SignalStrategy):
  def __init__(self):
    sma1, sma2 = bt.ind.SMA(period=10), bt.ind.SMA(period=30)
    crossover = bt.ind.CrossOver(sma1, sma2)
    self.signal_add(bt.SIGNAL_LONG, crossover)


cerebro = bt.Cerebro()
cerebro.addstrategy(SmaCross)

store = alpaca_backtrader_api.AlpacaStore(
    key_id=ALPACA_API_KEY,
    secret_key=ALPACA_SECRET_KEY,
    paper=ALPACA_PAPER
)

if not ALPACA_PAPER:
  broker = store.getbroker()  # or just alpaca_backtrader_api.AlpacaBroker()
  cerebro.setbroker(broker)

DataFactory = store.getdata  # or use alpaca_backtrader_api.AlpacaData
data0 = DataFactory(dataname='AAPL', historical=True, fromdate=datetime(
    2021, 1, 1), timeframe=bt.TimeFrame.Days)
cerebro.adddata(data0)

print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())
cerebro.run()
print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())
cerebro.plot(iplot=False)

In [7]:
import alpaca_backtrader_api
import backtrader as bt
from datetime import datetime

# Your credentials here
ALPACA_API_KEY = "PK8SSSXD3FX164IYREJ5"
ALPACA_SECRET_KEY = "NPKEgtQyt3fsRodbOqn4d46V19XI5W9S8Cv64dDT"

"""
3 options:
 - backtest (IS_BACKTEST=True, IS_LIVE=False)
 - paper trade (IS_BACKTEST=False, IS_LIVE=False)
 - live trade (IS_BACKTEST=False, IS_LIVE=True)
"""
IS_BACKTEST = True
IS_LIVE = False
symbol = "TSLA"

class SessionFilter(object):
    def __init__(self, data):
        pass
    def __call__(self, data):
        if data.p.sessionstart <= data.datetime.time() <= data.p.sessionend:
            return False  
        data.backwards()  
        return True 
class SmaCross1(bt.Strategy):
    # list of parameters which are configurable for the strategy
    params = dict(
        pfast=5,  # period for the fast moving average
        pslow=15,   # period for the slow moving average
        rsi_per=14,
        rsi_upper=60.0,
        rsi_lower=40.0,
        rsi_out=50.0,
        warmup=35
    )

    def log(self, txt, dt=None):
        dt = dt or self.data.datetime[0]
        dt = bt.num2date(dt)
        print('%s, %s' % (dt.isoformat(), txt))

    def notify_trade(self, trade):
        self.log("placing trade for {}. target size: {}".format(
            trade.getdataname(),
            trade.size))

    def notify_order(self, order):
        print(f"Order notification. status{order.getstatusname()}.")
        print(f"Order info. status{order.info}.")

    def notify_store(self, msg, *args, **kwargs):
        super().notify_store(msg, *args, **kwargs)
        self.log(msg)

    def stop(self):
        print('==================================================')
        print('Starting Value - %.2f' % self.broker.startingcash)
        print('Ending   Value - %.2f' % self.broker.getvalue())
        print('==================================================')

    def __init__(self):
        self.live_bars = False
        sma1 = bt.ind.SMA(self.data0, period=self.p.pfast)
        sma2 = bt.ind.SMA(self.data0, period=self.p.pslow)
        self.crossover = bt.ind.CrossOver(sma1, sma2,plotskip=True)

        rsi = bt.indicators.RSI(period=self.p.rsi_per,
                                upperband=self.p.rsi_upper,
                                lowerband=self.p.rsi_lower,plotskip=True)

        self.crossdown = bt.ind.CrossDown(rsi, self.p.rsi_upper,plotskip=True)
        self.crossup = bt.ind.CrossUp(rsi, self.p.rsi_lower,plotskip=True)

    def notify_data(self, data, status, *args, **kwargs):
        super().notify_data(data, status, *args, **kwargs)
        print('*' * 5, 'DATA NOTIF:', data._getstatusname(status), *args)
        if data._getstatusname(status) == "LIVE":
            self.live_bars = True

    def next(self):
        if not self.live_bars and not IS_BACKTEST:
            # only run code if we have live bars (today's bars).
            # ignore if we are backtesting
            return
        # if fast crosses slow to the upside
        if not self.positionsbyname[symbol].size:
            if self.crossover > 0 or self.crossup > 0:
                self.buy(data=data0, size=80)  # enter long

        # in the market & cross to the downside
        if self.positionsbyname[symbol].size:
            if self.crossover <= 0 or self.crossdown < 0:
                self.close(data=data0)  # close long position


if __name__ == '__main__':
    import logging
    logging.basicConfig(format='%(asctime)s %(message)s', level=logging.INFO)
    cerebro = bt.Cerebro()
    cerebro.addstrategy(SmaCross1)
    store = alpaca_backtrader_api.AlpacaStore(
        key_id=ALPACA_API_KEY,
        secret_key=ALPACA_SECRET_KEY,
        paper=not IS_LIVE,
    )

    DataFactory = store.getdata  # or use alpaca_backtrader_api.AlpacaData
    if IS_BACKTEST:
        data0 = DataFactory(dataname=symbol, historical=True,
                            fromdate=datetime(2021, 11, 1,9,30),
                            todate=datetime(2021, 11, 15,9,30),
                            timeframe=bt.TimeFrame.Minutes,
                            data_feed='CTA')

    else:
        data0 = DataFactory(dataname=symbol,
                            historical=False,
                            timeframe=bt.TimeFrame.Ticks,
                            backfill_start=False,
                            data_feed='CTA')
        # or just alpaca_backtrader_api.AlpacaBroker()
        broker = store.getbroker()
        cerebro.setbroker(broker)
    data0.addfilter(SessionFilter)
    cerebro.adddata(data0)
    cerebro.addanalyzer(bt.analyzers.SharpeRatio, _name = 'SharpeRatio')
    cerebro.addanalyzer(bt.analyzers.DrawDown, _name='DW')
    if IS_BACKTEST:
        # backtrader broker set initial simulated cash
        cerebro.broker.setcash(100000.0)

    #cerebro.run()
    result=cerebro.run()
    #strat=result[0]
    #print('Sharpe ratio:', strat.analyzers.SharpeRatio.get_analysis())
    cerebro.plot(iplot=False)

***** DATA NOTIF: DELAYED
Order notification. statusSubmitted.
Order info. statusAutoOrderedDict().
Order notification. statusAccepted.
Order info. statusAutoOrderedDict().
Order notification. statusCompleted.
Order info. statusAutoOrderedDict().
2021-11-01T10:34:00, placing trade for TSLA. target size: 80
Order notification. statusSubmitted.
Order info. statusAutoOrderedDict().
Order notification. statusAccepted.
Order info. statusAutoOrderedDict().
Order notification. statusCompleted.
Order info. statusAutoOrderedDict().
2021-11-01T10:36:00, placing trade for TSLA. target size: 0
Order notification. statusSubmitted.
Order info. statusAutoOrderedDict().
Order notification. statusAccepted.
Order info. statusAutoOrderedDict().
Order notification. statusCompleted.
Order info. statusAutoOrderedDict().
2021-11-01T11:28:00, placing trade for TSLA. target size: 80
Order notification. statusSubmitted.
Order info. statusAutoOrderedDict().
Order notification. statusAccepted.
Order info. statusA

Order notification. statusSubmitted.
Order info. statusAutoOrderedDict().
Order notification. statusAccepted.
Order info. statusAutoOrderedDict().
Order notification. statusCompleted.
Order info. statusAutoOrderedDict().
2021-11-01T14:23:00, placing trade for TSLA. target size: 0
Order notification. statusSubmitted.
Order info. statusAutoOrderedDict().
Order notification. statusAccepted.
Order info. statusAutoOrderedDict().
Order notification. statusCompleted.
Order info. statusAutoOrderedDict().
2021-11-01T15:40:00, placing trade for TSLA. target size: 80
Order notification. statusSubmitted.
Order info. statusAutoOrderedDict().
Order notification. statusAccepted.
Order info. statusAutoOrderedDict().
Order notification. statusCompleted.
Order info. statusAutoOrderedDict().
2021-11-01T15:41:00, placing trade for TSLA. target size: 0
Order notification. statusSubmitted.
Order info. statusAutoOrderedDict().
Order notification. statusAccepted.
Order info. statusAutoOrderedDict().
Order not

Order notification. statusSubmitted.
Order info. statusAutoOrderedDict().
Order notification. statusAccepted.
Order info. statusAutoOrderedDict().
Order notification. statusCompleted.
Order info. statusAutoOrderedDict().
2021-11-02T14:45:00, placing trade for TSLA. target size: 80
Order notification. statusSubmitted.
Order info. statusAutoOrderedDict().
Order notification. statusAccepted.
Order info. statusAutoOrderedDict().
Order notification. statusCompleted.
Order info. statusAutoOrderedDict().
2021-11-02T14:46:00, placing trade for TSLA. target size: 0
Order notification. statusSubmitted.
Order info. statusAutoOrderedDict().
Order notification. statusAccepted.
Order info. statusAutoOrderedDict().
Order notification. statusCompleted.
Order info. statusAutoOrderedDict().
2021-11-02T14:47:00, placing trade for TSLA. target size: 80
Order notification. statusSubmitted.
Order info. statusAutoOrderedDict().
Order notification. statusAccepted.
Order info. statusAutoOrderedDict().
Order no

Order notification. statusSubmitted.
Order info. statusAutoOrderedDict().
Order notification. statusAccepted.
Order info. statusAutoOrderedDict().
Order notification. statusCompleted.
Order info. statusAutoOrderedDict().
2021-11-03T12:49:00, placing trade for TSLA. target size: 80
Order notification. statusSubmitted.
Order info. statusAutoOrderedDict().
Order notification. statusAccepted.
Order info. statusAutoOrderedDict().
Order notification. statusCompleted.
Order info. statusAutoOrderedDict().
2021-11-03T12:50:00, placing trade for TSLA. target size: 0
Order notification. statusSubmitted.
Order info. statusAutoOrderedDict().
Order notification. statusAccepted.
Order info. statusAutoOrderedDict().
Order notification. statusCompleted.
Order info. statusAutoOrderedDict().
2021-11-03T12:56:00, placing trade for TSLA. target size: 80
Order notification. statusSubmitted.
Order info. statusAutoOrderedDict().
Order notification. statusAccepted.
Order info. statusAutoOrderedDict().
Order no

Order notification. statusSubmitted.
Order info. statusAutoOrderedDict().
Order notification. statusAccepted.
Order info. statusAutoOrderedDict().
Order notification. statusCompleted.
Order info. statusAutoOrderedDict().
2021-11-04T09:32:00, placing trade for TSLA. target size: 80
Order notification. statusSubmitted.
Order info. statusAutoOrderedDict().
Order notification. statusAccepted.
Order info. statusAutoOrderedDict().
Order notification. statusCompleted.
Order info. statusAutoOrderedDict().
2021-11-04T09:37:00, placing trade for TSLA. target size: 0
Order notification. statusSubmitted.
Order info. statusAutoOrderedDict().
Order notification. statusAccepted.
Order info. statusAutoOrderedDict().
Order notification. statusCompleted.
Order info. statusAutoOrderedDict().
2021-11-04T10:32:00, placing trade for TSLA. target size: 80
Order notification. statusSubmitted.
Order info. statusAutoOrderedDict().
Order notification. statusAccepted.
Order info. statusAutoOrderedDict().
Order no

...

Order notification. statusSubmitted.
Order info. statusAutoOrderedDict().
Order notification. statusAccepted.
Order info. statusAutoOrderedDict().
Order notification. statusCompleted.
Order info. statusAutoOrderedDict().
2021-11-10T12:08:00, placing trade for TSLA. target size: 0
Order notification. statusSubmitted.
Order info. statusAutoOrderedDict().
Order notification. statusAccepted.
Order info. statusAutoOrderedDict().
Order notification. statusCompleted.
Order info. statusAutoOrderedDict().
2021-11-10T12:09:00, placing trade for TSLA. target size: 80
Order notification. statusSubmitted.
Order info. statusAutoOrderedDict().
Order notification. statusAccepted.
Order info. statusAutoOrderedDict().
Order notification. statusCompleted.
Order info. statusAutoOrderedDict().
2021-11-10T12:10:00, placing trade for TSLA. target size: 0
Order notification. statusSubmitted.
Order info. statusAutoOrderedDict().
Order notification. statusAccepted.
Order info. statusAutoOrderedDict().
Order not

Order notification. statusSubmitted.
Order info. statusAutoOrderedDict().
Order notification. statusAccepted.
Order info. statusAutoOrderedDict().
Order notification. statusCompleted.
Order info. statusAutoOrderedDict().
2021-11-10T15:31:00, placing trade for TSLA. target size: 0
Order notification. statusSubmitted.
Order info. statusAutoOrderedDict().
Order notification. statusAccepted.
Order info. statusAutoOrderedDict().
Order notification. statusCompleted.
Order info. statusAutoOrderedDict().
2021-11-10T15:57:00, placing trade for TSLA. target size: 80
Order notification. statusSubmitted.
Order info. statusAutoOrderedDict().
Order notification. statusAccepted.
Order info. statusAutoOrderedDict().
Order notification. statusCompleted.
Order info. statusAutoOrderedDict().
2021-11-10T15:58:00, placing trade for TSLA. target size: 0
Order notification. statusSubmitted.
Order info. statusAutoOrderedDict().
Order notification. statusAccepted.
Order info. statusAutoOrderedDict().
Order not

Order info. statusAutoOrderedDict().
Order notification. statusAccepted.
Order info. statusAutoOrderedDict().
Order notification. statusCompleted.
Order info. statusAutoOrderedDict().
2021-11-11T13:54:00, placing trade for TSLA. target size: 0
Order notification. statusSubmitted.
Order info. statusAutoOrderedDict().
Order notification. statusAccepted.
Order info. statusAutoOrderedDict().
Order notification. statusCompleted.
Order info. statusAutoOrderedDict().
2021-11-11T14:06:00, placing trade for TSLA. target size: 80
Order notification. statusSubmitted.
Order info. statusAutoOrderedDict().
Order notification. statusAccepted.
Order info. statusAutoOrderedDict().
Order notification. statusCompleted.
Order info. statusAutoOrderedDict().
2021-11-11T14:07:00, placing trade for TSLA. target size: 0
Order notification. statusSubmitted.
Order info. statusAutoOrderedDict().
Order notification. statusAccepted.
Order info. statusAutoOrderedDict().
Order notification. statusCompleted.
Order inf

Order notification. statusSubmitted.
Order info. statusAutoOrderedDict().
Order notification. statusAccepted.
Order info. statusAutoOrderedDict().
Order notification. statusCompleted.
Order info. statusAutoOrderedDict().
2021-11-12T13:27:00, placing trade for TSLA. target size: 80
Order notification. statusSubmitted.
Order info. statusAutoOrderedDict().
Order notification. statusAccepted.
Order info. statusAutoOrderedDict().
Order notification. statusCompleted.
Order info. statusAutoOrderedDict().
2021-11-12T13:28:00, placing trade for TSLA. target size: 0
Order notification. statusSubmitted.
Order info. statusAutoOrderedDict().
Order notification. statusAccepted.
Order info. statusAutoOrderedDict().
Order notification. statusCompleted.
Order info. statusAutoOrderedDict().
2021-11-12T13:45:00, placing trade for TSLA. target size: 80
Order notification. statusSubmitted.
Order info. statusAutoOrderedDict().
Order notification. statusAccepted.
Order info. statusAutoOrderedDict().
Order no

C:\conda\envs\wsb\lib\site-packages\ipykernel\eventloops.py:256: RuntimeWarning: coroutine 'Kernel.do_one_iteration' was never awaited
  self.func()


In [ ]:
import alpaca_backtrader_api
import backtrader as bt
from datetime import datetime

# Your credentials here
ALPACA_API_KEY = "PK8SSSXD3FX164IYREJ5"
ALPACA_SECRET_KEY = "NPKEgtQyt3fsRodbOqn4d46V19XI5W9S8Cv64dDT"


"""
You have 3 options:
 - backtest (IS_BACKTEST=True, IS_LIVE=False)
 - paper trade (IS_BACKTEST=False, IS_LIVE=False)
 - live trade (IS_BACKTEST=False, IS_LIVE=True)
"""
IS_BACKTEST = True
IS_LIVE = False
symbol = "QQQ"
class SessionFilter(object):
    def __init__(self, data):
        pass

    def __call__(self, data):
        if data.p.sessionstart <= data.datetime.time() <= data.p.sessionend:
            # bar is in the session
            return False  # tell outer data loop the bar can be processed

        # bar outside of the regular session times
        data.backwards()  # remove bar from data stack
        return True 

class SmaCross(bt.SignalStrategy):
    def __init__(self):
        sma1, sma2 = bt.ind.SMA(period=3,plotskip=True), bt.ind.SMA(period=10,plotskip=True)
        crossover = bt.ind.CrossOver(sma1, sma2)
        self.signal_add(bt.SIGNAL_LONG, crossover)


if __name__ == '__main__':
    cerebro = bt.Cerebro()
    cerebro.addstrategy(SmaCross)

    store = alpaca_backtrader_api.AlpacaStore(
        key_id=ALPACA_API_KEY,
        secret_key=ALPACA_SECRET_KEY,
        paper=not IS_LIVE,
    )

    DataFactory = store.getdata  # or use alpaca_backtrader_api.AlpacaData
    if IS_BACKTEST:
        data0 = DataFactory(dataname=symbol, historical=True,
                            fromdate=datetime(2021, 10, 1,9,30),
                            todate=datetime(2021, 11, 15,9,30),
                            timeframe=bt.TimeFrame.Minutes,
                            data_feed='CTA')
        #print(len(data0))
    else:
        data0 = DataFactory(dataname=symbol,
                            historical=False,
                            timeframe=bt.TimeFrame.Days,
                            data_feed='CTA')
        # or just alpaca_backtrader_api.AlpacaBroker()
        broker = store.getbroker()
        cerebro.setbroker(broker)
    data0.addfilter(SessionFilter)
    cerebro.adddata(data0)

    print('Starting Portfolio Value: {}'.format(cerebro.broker.getvalue()))
    cerebro.run()
    print('Final Portfolio Value: {}'.format(cerebro.broker.getvalue()))
    cerebro.plot(iplot=False)